In [ ]:
pip install GitPython

In [ ]:
pip install JPype1

In [ ]:
def printStatus(index, size):
    print("{0}% Completed samples".format((index / size) * 100))

In [ ]:
from git import Repo
import pandas as pd
import numpy as np
import os
import fnmatch
from statistics import mean
import shutil

In [ ]:
def findPaths(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        if '.git' in root:
            continue
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

Buse's Readability

In [ ]:
# Boiler plate stuff to start the module
import jpype
from jpype import *

class BuseReadability(object):
    def __init__(self, project):
#         super(Readability, self).__init__()
        self.project = project
        self.startJVM()
        self.readabilityPackage = JPackage("raykernel").apps.readability.eval.Main
        self.repositoryPath = "repositories\\"+project

    def startJVM(self):
        if not jpype.isJVMStarted():
            jpype.startJVM(jpype.getDefaultJVMPath(), '-Djava.class.path=readability.jar',  '-ea', '-Xmx4096M', convertStrings=True)

    def shutdownJVM(self):
        jpype.shutdownJVM()

    def getReadability(self):
        javaFiles = findPaths("*.java", self.repositoryPath)
        if len(javaFiles) == 0: return 0
        readability = 0
        array = []
        for javaFile in javaFiles:
            try:
                file = open(javaFile)
                file = file.read()
            except:
                file = ""
            array.append(float(self.readabilityPackage.getReadability(file)))
        return mean(array)

In [ ]:
def getBuseReadability(sample, commit):
    checkoutTo(sample, commit)
    r = BuseReadability(sample.split("/")[0]+"\\"+sample.split("/")[1])
    readability = r.getReadability()
    del r
    return readability

In [ ]:
def checkoutTo(project, sha):
    projectPath = "repositories/" + project
    repository = Repo(projectPath)
    repository.git.checkout(sha, "-f")

In [ ]:
samples = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")

In [ ]:
samples["full_name"] = samples["framework"] + "/" + samples["path"]

In [ ]:
samples = samples["full_name"].unique().tolist()

In [ ]:
samples = samples[samples.index("googlesamples/android-testdpc"):]

In [ ]:
for sample in samples:
    print(sample)
    file_address = f"metrics/{sample}.csv"
    metrics = pd.read_csv(file_address)
    metrics["Buse Readability"] = 0
    last_commit_result = ("0",0)
    printStatus(samples.index(sample), len(samples))
    for index in range(0, len(metrics)):
        print(f"{index}/{len(metrics)} of line analized from {sample}")
        current_commit = metrics.iloc[index]["commitSha"]
        if not current_commit == last_commit_result[0]:
            last_commit_result = (current_commit, getBuseReadability(sample, current_commit))
        metrics.iloc[index, len(metrics.columns.values) - 1] = last_commit_result[1]
        metrics.to_csv(file_address, index=False)